# Week 3 Part 1 
# Explore and cluster the neighborhoods in Toronto

In [1]:
import pandas as pd

In [2]:
#!wget -q -O "PostalCodesCanada.html" "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" 
#print("File Downloaded!")
#PCC_df = pd.read_html("PostalCodesCanada.html")
#PCC_df

In [6]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.2            |           py36_0          61 KB  anaconda
    beautifulsoup4-4.8.0       |           py36_0         147 KB  anaconda
    ------------------------------------------------------------
                                           Total:         208 KB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.2-py36_0 anaconda

The following packages will be UPDATED:

    beautifulsoup4: 4.6.3-py37_0          --> 4.8.0-py36_0 anaconda


soupsieve-1.9.2      | 61 KB     | 

In [4]:
conda install -c anaconda lxml

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB  anaconda
    openssl-1.0.2s             |       h7b6447c_0         3.1 MB  anaconda
    certifi-2019.6.16          |           py36_1         156 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.8 MB

The following packages will be UPDATED:

    certifi: 2019.6.16-py36_1     conda-forge --> 2019.6.16-py36_1     anaconda
    lxml:    4.2.5-py37hefd8a0e_0             --> 

In [7]:
from bs4 import BeautifulSoup
import urllib.request as urlreq
import lxml
print('done')

done


In [8]:
with urlreq.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") as response:
    PCC_html = response.read()

In [9]:
#print(PCC_html)

In [10]:
soup = BeautifulSoup(PCC_html, 'html.parser')
#print(soup.prettify())

In [11]:
#soup.table
table = soup.find_all('table')[0] # finding the first table in the html
df = pd.read_html(str(table))[0] # conversion of html to string and then conversion to dataframes
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
df_with_borough = df[df['Borough'] != "Not assigned"] #  Ignore cells with a borough that is Not assigned
df_with_borough.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [13]:
df_with_borough.reset_index(inplace=True, drop=True)
df_with_borough.head(7)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned


In [14]:
df_same_borough_nbr = df_with_borough

#  Ignore cells with a borough that is Not assigned
p = df_same_borough_nbr.loc[df_same_borough_nbr['Neighbourhood'] == "Not assigned"]
p.head(7)

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


In [15]:
# copying Neighbourhood with Borough that has "Not assigned"
df_same_borough_nbr.iloc[p.index[0]]['Neighbourhood'] = df_same_borough_nbr.iloc[p.index[0]]['Borough']
df_same_borough_nbr.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [16]:
df_same_borough_nbr.describe()

,Postcode,Borough,Neighbourhood
count,211,211,211
unique,103,11,209
top,M8Y,Etobicoke,Runnymede
freq,8,45,2


In [17]:
# Checking group of data by Postcode
pp=df_same_borough_nbr.groupby('Postcode')
k=pp.get_group('M5A')
k

,Postcode,Borough,Neighbourhood
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park


In [18]:
# Creating new dataframe
newDF = pd.DataFrame(columns = ['Postcode', 'Borough', 'Neighbourhood'])

In [19]:

# getting unique post codes for iteration to all the groups based on 'postcode' one by one for extracting neighbourhood
df_uniquePostcodes = df_same_borough_nbr['Postcode'].unique()

for PC in df_uniquePostcodes:
    Total_Nbr_of_PC = pp.get_group(PC)
    
    # copying the rows with no multiple Neighbourhoods
    if len(Total_Nbr_of_PC)==1:
        POSTCODE = Total_Nbr_of_PC.iloc[0,0]
        BOROUGH = Total_Nbr_of_PC.iloc[0,1]
        NEIGHBOURHOOD = Total_Nbr_of_PC.iloc[0,2]
        newDF = newDF.append(pd.DataFrame({'Postcode':POSTCODE,'Borough':BOROUGH, 'Neighbourhood':NEIGHBOURHOOD}, index=[0]) , ignore_index=True)
    elif len(Total_Nbr_of_PC)>1:
        # Following line combines all the Neighbourhoods of a group in rows seperated with comma
        tt = Total_Nbr_of_PC[['Neighbourhood']].apply(lambda alpha: ','.join(alpha),axis=0)
        POSTCODE = Total_Nbr_of_PC.iloc[0,0]
        BOROUGH = Total_Nbr_of_PC.iloc[0,1]
        NEIGHBOURHOOD = tt[0]
        
        #print(Total_Nbr_of_PC,', Length = ', len(Total_Nbr_of_PC))
        #print('Postcode = ', Total_Nbr_of_PC.iloc[0,0])
        #print('Borough = ', Total_Nbr_of_PC.iloc[0,1])
        #print('Appended Nbr = ', tt[0], '\n')
        
        newDF = newDF.append(pd.DataFrame({'Postcode':POSTCODE,'Borough':BOROUGH, 'Neighbourhood':NEIGHBOURHOOD}, index=[0]) , ignore_index=True)

    #print(pp.get_group(PC))
    #print(PC)
    
newDF.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [20]:
newDF.shape

(103, 3)

In [21]:
newDF.describe()

,Postcode,Borough,Neighbourhood
count,103,103,103
unique,103,11,103
top,M4G,North York,"Little Portugal,Trinity"
freq,1,24,1


In [22]:
newDF.to_csv('combinedNeighbourhoodData.csv', index=False)